In [ ]:
# 维度情感分类
# !pip install paddlenlp==2.4.2
!pip show paddlenlp

In [1]:
#训练
import os
from functools import partial
import paddle
import paddle.nn.functional as F
from paddlenlp.datasets import load_dataset
from paddlenlp.data import Pad, Stack, Tuple
from visualdl import LogWriter
from paddlenlp.transformers import BertTokenizer, BertForSequenceClassification, LinearDecayWithWarmup
from paddlenlp.transformers import SkepTokenizer, SkepModel, SkepForSequenceClassification
from paddlenlp.transformers import ErnieTokenizer, ErnieForSequenceClassification
from paddlenlp.transformers import RobertaTokenizer, RobertaForSequenceClassification
from utils import  data_cls

paddle.set_device("gpu:0")
train_path = "model_train.txt"
label_path = "model_label_cls.dict"
label2id, id2label = data_cls.load_dict(label_path)
num_epoch = 20
max_seq_len = 512
weight_decay = 0.01
warmup_proportion = 0
max_grad_norm = 1.0
log_step = 20
seed = 1000

batch_size =32
learning_rate = 1e-04

model_name = "macbert-large-chinese"
checkpoint = "./checkpoint/checkpoint_bert_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/"
writer = LogWriter(logdir="./log/log_bert_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/")
model = BertForSequenceClassification.from_pretrained(model_name,num_classes=len(label2id))
tokenizer = BertTokenizer.from_pretrained(model_name)

# model_name = "hfl/roberta-wwm-ext-large"
# checkpoint = "./checkpoint/checkpoint_roberta_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/"
# writer = LogWriter(logdir="./log/log_roberta_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/")
# model = RobertaForSequenceClassification.from_pretrained(model_name,num_classes=len(label2id))
# tokenizer = BertTokenizer.from_pretrained(model_name)
#
# model_name =  "ernie-3.0-xbase-zh"
# checkpoint = "./checkpoint/checkpoint_ernie_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/"
# writer = LogWriter(logdir="./log/log_ernie_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/")
# model = ErnieForSequenceClassification(model_name, num_classes=len(label2id))
# tokenizer = ErnieTokenizer.from_pretrained(model_name)
#
# model_name = "skep_ernie_1.0_large_ch"
# checkpoint = "./checkpoint/checkpoint_skep_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/"
# writer = LogWriter(logdir="./log/log_skep_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/")
# model = SkepForSequenceClassification.from_pretrained(model_name,num_classes=len(label2id))
# tokenizer = SkepTokenizer.from_pretrained(model_name)

if not os.path.exists(checkpoint):
    os.mkdir(checkpoint)

train_ds = load_dataset(data_cls.read, data_path=train_path, lazy=False)
trans_func = partial(data_cls.convert_example_to_feature, tokenizer=tokenizer, label2id=label2id, max_seq_len=max_seq_len)
train_ds = train_ds.map(trans_func, lazy=False)
batchify_fn = lambda samples, fn=Tuple(Pad(axis=0, pad_val=tokenizer.pad_token_id), Pad(axis=0, pad_val=tokenizer.pad_token_type_id),Stack(dtype="int64"), Stack(dtype="int64")): fn(samples)
train_batch_sampler = paddle.io.BatchSampler(train_ds, batch_size=batch_size, shuffle=True)
train_loader = paddle.io.DataLoader(train_ds, batch_sampler=train_batch_sampler, collate_fn=batchify_fn)
num_training_steps = len(train_loader) * num_epoch
lr_scheduler = LinearDecayWithWarmup(learning_rate=learning_rate, total_steps=num_training_steps, warmup=warmup_proportion)
decay_params = [p.name for n, p in model.named_parameters() if not any(nd in n for nd in ["bias", "norm"])]
grad_clip = paddle.nn.ClipGradByGlobalNorm(max_grad_norm)
optimizer = paddle.optimizer.AdamW(learning_rate=lr_scheduler, parameters=model.parameters(), weight_decay=weight_decay, apply_decay_param_fun=lambda x: x in decay_params, grad_clip=grad_clip)

def train():
    global_step, best_res = 1, 0.
    model.train()
    for epoch in range(1, num_epoch+1):
        for batch_data in train_loader():
            input_ids, token_type_ids, _, labels = batch_data
            logits = model(input_ids, token_type_ids=token_type_ids)   
            loss = F.cross_entropy(logits, labels, soft_label=True)
            writer.add_scalar(tag="train/loss", step=global_step, value=loss)
            loss.backward()
            lr_scheduler.step()
            optimizer.step()
            optimizer.clear_grad()
            if global_step > 0 and global_step % log_step == 0:
                print(f"epoch: {epoch} - global_step: {global_step}/{num_training_steps} - loss:{loss.numpy().item():.6f}")
            global_step += 1
    paddle.save(model.state_dict(), f"{checkpoint}/best_cls.pdparams")      
    print(batch_size,learning_rate,model_name)
train()

[2023-08-14 10:46:32,725] [    INFO] - Already cached C:\Users\93461\.paddlenlp\models\macbert-large-chinese\bert-base-chinese-vocab.txt
[2023-08-14 10:46:32,738] [    INFO] - tokenizer config file saved in C:\Users\93461\.paddlenlp\models\macbert-large-chinese\tokenizer_config.json
[2023-08-14 10:46:32,739] [    INFO] - Special tokens file saved in C:\Users\93461\.paddlenlp\models\macbert-large-chinese\special_tokens_map.json
[2023-08-14 10:46:32,743] [    INFO] - Already cached C:\Users\93461\.paddlenlp\models\macbert-large-chinese\macbert-large-chinese.pdparams


SystemError: (Fatal) Operator matmul_v2 raises an struct paddle::memory::allocation::BadAlloc exception.
The exception content is
:ResourceExhaustedError: 

Out of memory error on GPU 0. Cannot allocate 16.892578MB memory on GPU 0, 5.999512GB memory has been allocated and available memory is only 0.000000B.

Please check whether there is any other process using GPU 0.
1. If yes, please stop them, or start PaddlePaddle on another GPU.
2. If no, please decrease the batch size of your model. 

 (at ..\paddle\fluid\memory\allocation\cuda_allocator.cc:87)
. (at ..\paddle\fluid\imperative\tracer.cc:307)


In [4]:
#验证
from functools import partial
import paddle
import numpy as np
from paddlenlp.datasets import load_dataset
from paddlenlp.data import Pad, Stack, Tuple
from paddlenlp.transformers import BertTokenizer, BertForSequenceClassification, LinearDecayWithWarmup
from paddlenlp.transformers import SkepTokenizer, SkepModel
from paddlenlp.transformers import ErnieTokenizer, ErnieForSequenceClassification
from paddlenlp.transformers import RobertaTokenizer, RobertaForSequenceClassification
from utils import data_cls

paddle.set_device("gpu:0")
test_path =r'model_test.txt'
label_path = "model_label_cls.dict"
label2id, id2label = data_cls.load_dict(label_path)
max_seq_len=512
att=0

batch_size = 16
learning_rate = 1e-05

model_name = "macbert-large-chinese"
checkpoint = "./checkpoint/checkpoint_bert_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/"
writer = LogWriter(logdir="./log/log_bert_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/")
model = BertForSequenceClassification.from_pretrained(model_name,num_classes=len(label2id))
tokenizer = BertTokenizer.from_pretrained(model_name)

# model_name = "hfl/roberta-wwm-ext-large"
# checkpoint = "./checkpoint/checkpoint_roberta_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/"
# writer = LogWriter(logdir="./log/log_roberta_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/")
# model = RobertaForSequenceClassification.from_pretrained(model_name,num_classes=len(label2id))
# tokenizer = BertTokenizer.from_pretrained(model_name)
#
# model_name =  "ernie-3.0-xbase-zh"
# checkpoint = "./checkpoint/checkpoint_ernie_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/"
# writer = LogWriter(logdir="./log/log_ernie_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/")
# model = ErnieForSequenceClassification(model_name, num_classes=len(label2id))
# tokenizer = ErnieTokenizer.from_pretrained(model_name)
#
# model_name = "skep_ernie_1.0_large_ch"
# checkpoint = "./checkpoint/checkpoint_skep_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/"
# writer = LogWriter(logdir="./log/log_skep_"+str(batch_size)+'_'+str(learning_rate)[-1]+"/")
# model = SkepForSequenceClassification.from_pretrained(model_name,num_classes=len(label2id))
# tokenizer = SkepTokenizer.from_pretrained(model_name)

with open(test_path,'r',encoding='utf-8') as f:
    data=f.read().split('\n')
model = BertForSequenceClassification.from_pretrained(model_name, num_classes=len(label2id))
loaded_state_dict = paddle.load(checkpoint+'best_cls.pdparams')
model.load_dict(loaded_state_dict)
model.eval()
test_ds =  load_dataset(data_cls.read, data_path=test_path, lazy=False)
trans_func = partial(data_cls.convert_example_to_feature, tokenizer=tokenizer, label2id=label2id, max_seq_len=max_seq_len)
test_ds = test_ds.map(trans_func, lazy=False)
batchify_fn = lambda samples, fn=Tuple(
        Pad(axis=0, pad_val=tokenizer.pad_token_id),
        Pad(axis=0, pad_val=tokenizer.pad_token_type_id),
        Stack(dtype="int64"),
        Stack(dtype="int64")
    ): fn(samples)
test_batch_sampler = paddle.io.BatchSampler(test_ds, batch_size=batch_size, shuffle=False)
test_loader = paddle.io.DataLoader(test_ds, batch_sampler=test_batch_sampler, collate_fn=batchify_fn)

label_true=[]
label_pred=[]

for idx, batch_data in enumerate(test_loader):
    input_ids, token_type_ids, seq_lens, labels = batch_data
    logits = model(input_ids, token_type_ids=token_type_ids)
    probs = F.softmax(logits, axis=1)
    probs = probs.numpy()
    preds = np.argmax(probs, axis=1)
    label_pred.extend(preds.tolist())
    label_true.extend(labels.numpy().tolist())
from sklearn.metrics import classification_report
print(classification_report(label_true, label_pred, digits=4))

def attention_shown():
    nums=-1
    a,b,c=data[nums].split('\t');sentence=b+'\t'+c
    lens=len(sentence)+2
    sum1=np.zeros([12,lens,lens])
    sum1=paddle.to_tensor(sum1)
    for m in range(0,12):
        for i in range(0,lens):
            for j in range(0,lens):
                sum=0
                for a in range(0,16):
                    sum=sum+att[m][0][a][i][j]
                sum1[m,i,j]=float(sum)
    sum1=sum1.sum(axis=0)
    sum2=np.zeros([12,lens,lens])
    sum2=paddle.to_tensor(sum2)
    for m in range(12,24):
        for i in range(0,lens):
            for j in range(0,lens):
                sum=0
                for a in range(0,16):

                    sum=sum+att[m][0][a][i][j]

                sum2[m-12,i,j]=float(sum)
    sum2=sum2.sum(axis=0)
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt
    data_tu=[]*len(sum1)
    for i in range(0,len(sum1)):
        sumss=sum1[i].sum()
        l=[]
        for j in range(0,len(sum1)):
            l.append(float(sum1[i][j]/sumss))
        data_tu.append(l)
    data_tu=[]*len(sum2)
    for i in range(0,len(sum2)):
        sumss=sum2[i].sum()
        l=[]
        for j in range(0,len(sum2)):
            l.append(float(sum2[i][j]/sumss))
        data_tu.append(l)
    df=pd.DataFrame(data_tu,index=[i for i in range(0,len(data_tu))],columns=[i for i in range(0,len(data_tu))])
    sns.heatmap(df,  fmt="g", cmap='Accent_r')
    plt.show()

# attention_shown

NameError: name 'LogWriter' is not defined

In [ ]:
######### 读取模型，分类，写入
import paddle
from paddlenlp.datasets import load_dataset
from paddlenlp.transformers import SkepTokenizer, SkepModel
from utils import  data_cls
from functools import partial
import paddle.nn.functional as F
from paddlenlp.data import Pad, Stack, Tuple
import numpy as np
label_path = "model_label_cls.dict"
model_path = "./checkpoint_skep_"+str(16)+'_'+str(5)+"/best_cls.pdparams"
model_name = "skep_ernie_1.0_large_ch"
max_seq_len=512
label2id, id2label = data_cls.load_dict(label_path)
paddle.set_device("gpu:0")
tokenizer = SkepTokenizer.from_pretrained(model_name)
skep = SkepModel.from_pretrained(model_name)
att=0
trans_func = partial(data_cls.convert_example_to_feature, tokenizer=tokenizer, label2id=label2id, max_seq_len=max_seq_len)
batchify_fn = lambda samples, fn=Tuple(Pad(axis=0, pad_val=tokenizer.pad_token_id), Pad(axis=0, pad_val=tokenizer.pad_token_type_id),Stack(dtype="int64"), Stack(dtype="int64")): fn(samples)
class SkepForSequenceClassification(paddle.nn.Layer):
    def __init__(self, skep, num_classes=36, dropout=None):
        super(SkepForSequenceClassification, self).__init__()
        self.num_classes = num_classes
        self.skep = skep
        self.dropout = paddle.nn.Dropout(dropout if dropout is not None else self.skep.config["hidden_dropout_prob"])
        self.classifier = paddle.nn.Linear(self.skep.config["hidden_size"], num_classes)
    def forward(self, input_ids, token_type_ids=None, position_ids=None, attention_mask=None):
        global att
        _, pooled_output = self.skep(input_ids, token_type_ids=token_type_ids, position_ids=position_ids, attention_mask=attention_mask)
        pooled_output = self.dropout(pooled_output)
        logits = self.classifier(pooled_output)
        return logits
model = SkepForSequenceClassification(skep, num_classes=len(label2id))
# #############读取模型
loaded_state_dict = paddle.load(model_path)
model.load_dict(loaded_state_dict)
model.eval()
if not os.path.exists('维度-情感分类结果'):
    os.mkdir('维度-情感分类结果')
for pinpai in  ['ALBD','AND','AT','BNL','GRN','LN','QPL','TB']:
    print(pinpai)
    mm=0
    pred_path='关键词抽取结果/关键词抽取结果_'+pinpai+'.tsv'
    pred_ds = load_dataset(data_cls.read, data_path=pred_path, lazy=False,is_pred=True)
    pred_ds = pred_ds.map(trans_func, lazy=False)
    pred_batch_sampler = paddle.io.BatchSampler(pred_ds, batch_size=8, shuffle=False)
    pred_loader = paddle.io.DataLoader(pred_ds, batch_sampler=pred_batch_sampler, collate_fn=batchify_fn)
    with open('关键词抽取结果/关键词抽取结果_'+pinpai+'.tsv','r',encoding='utf-8') as f:
        data=f.readlines()
    with open('维度-情感分类结果/维度-情感分类结果_'+pinpai+'.tsv','w',encoding='utf-8') as f:
        i=0
        for batch_data in pred_loader():
            input_ids, token_type_ids, _ , label= batch_data
            logits = model(input_ids, token_type_ids=token_type_ids)
            probs = F.softmax(logits, axis=1)
            probs = probs.numpy()
            idx = np.argmax(probs, axis=1)
            idx = idx.tolist()
            for each in idx:
                f.write(str(each)+'\t'+data[i])
                i+=1;mm+=1
            if (mm%200==0 and mm>1) or mm==len(data):
                print('\rDone{0}%.({1}/{2})'.format(int(mm*100/len(data)),mm,len(data)),end='',flush=True)
        print(' ')

In [3]:
with open('model_label_cls.dict','r',encoding='utf-8') as f:
    data=f.readlines()
yingshe={}
[yingshe.update({str(i):data[i]}) for i in range(len(data))]
print(yingshe)
for pinpai in  ['ALBD','AND','AT','BNL','GRN','LN','QPL','TB']:
    with open('维度-情感分类结果/维度-情感分类结果_'+pinpai+'.tsv','r',encoding='utf-8') as f:
        data=f.readlines()
    with open('维度-情感分类结果/维度-情感分类结果2_'+pinpai+'.tsv','w',encoding='utf-8') as f:
        for each in data:
            w=each.split('\t')[0]
            w=yingshe[w][0:-1]
            f.write(w+'\t'+'\n')

{'0': '舒适1\n', '1': '耐久1\n', '2': '精细1\n', '3': '保障1\n', '4': '牌证1\n', '5': '外观1\n', '6': '外部1\n', '7': '响应及时性1\n', '8': '沟通有效性1\n', '9': '信息安全性1\n', '10': '补偿合理性1\n', '11': '服务专业性1\n', '12': '舒适2\n', '13': '耐久2\n', '14': '精细2\n', '15': '保障2\n', '16': '牌证2\n', '17': '外观2\n', '18': '外部2\n', '19': '响应及时性2\n', '20': '沟通有效性2\n', '21': '信息安全性2\n', '22': '补偿合理性2\n', '23': '服务专业性2\n', '24': '舒适3\n', '25': '耐久3\n', '26': '精细3\n', '27': '保障3\n', '28': '牌证3\n', '29': '外观3\n', '30': '外部3\n', '31': '响应及时性3\n', '32': '沟通有效性3\n', '33': '信息安全性3\n', '34': '补偿合理性3\n', '35': '服务专业性3\n'}
